In [ ]:
import cv2
import torch
import numpy as np
import open3d as o3d
from pathlib import Path

cams_limit = 100
rgb_dir = Path('EiffelTower/train/rgb/')

In [ ]:
inits = []
for i, rgb_path in enumerate(sorted(rgb_dir.iterdir())):
    init_path = rgb_path.parent / '..' / 'init' / rgb_path.with_suffix('.dat').name
    init = torch.load(init_path)
    init = init[:, torch.any(init != 0, dim=0)].T
    inits.append(init[::100])
inits = torch.vstack(inits)
inits = inits[torch.all(inits.abs() < 1000, dim=1)]  # filter outliers
all_pcl = o3d.geometry.PointCloud()
all_pcl.points = o3d.utility.Vector3dVector(inits.numpy())
all_pcl.paint_uniform_color([0.5, 0.5, 0.5])

In [ ]:
vis = o3d.visualization.Visualizer()
vis.create_window()

vis.add_geometry(all_pcl)

pcl = o3d.geometry.PointCloud()
vis.add_geometry(pcl)

cameras = []

for i, rgb_path in enumerate(sorted(rgb_dir.iterdir())):
    
    # update point cloud
    init_path = rgb_path.parent / '..' / 'init' / rgb_path.with_suffix('.dat').name
    init = torch.load(init_path)
    init = init[:, torch.any(init != 0, dim=0)].T
    
    pcl.points = o3d.utility.Vector3dVector(init.numpy())
    pcl.paint_uniform_color([1.0, 0.0, 0.0])
    vis.update_geometry(pcl)
    
    # set up camera
    rgb = cv2.imread(str(rgb_path))
    h, w, _ = rgb.shape
    
    pose_path = rgb_path.parent / '..' / 'poses' / rgb_path.with_suffix('.txt').name
    pose = np.stack([np.array(list(map(float, line.split(' ')))) for line in pose_path.read_text().splitlines()])
    
    focal_path = rgb_path.parent / '..' / 'calibration' / rgb_path.with_suffix('.txt').name
    focal = float(focal_path.read_text())
    
    K = np.array([
        [focal, 0, w / 2],
        [0, focal, h / 2],
        [0, 0, 1]
    ])
    
    camera = o3d.geometry.LineSet.create_camera_visualization(w, h, K, np.linalg.inv(pose), 1.0)
    camera.paint_uniform_color([1.0, 0.0, 0.0])
    vis.add_geometry(camera, reset_bounding_box=True if i < cams_limit else False)
    cameras.append(camera)
    if len(cameras) > cams_limit:
        vis.remove_geometry(cameras[0], reset_bounding_box=False)
        del cameras[0]
    for camera in cameras[:-1]:
        camera.paint_uniform_color([1.0, 0.9, 0.9])
        vis.update_geometry(camera)

    vis.poll_events()
    vis.update_renderer()
    
vis.destroy_window()